In [3]:
from types import NoneType

from telethon.sync import TelegramClient as TC
from telethon import TelegramClient
from telethon.tl.functions.channels import JoinChannelRequest
import telethon
import json
from PIL import Image
from io import BytesIO
import networkx as nx

In [4]:
credentials = json.load(open('../../keys/telegram_key.json'))
client_id = credentials['api_id']
client_secret = credentials['api_hash']

In [5]:
# Costruisco oggetto TelegramClient

client = TelegramClient('anon', client_id, client_secret)

In [6]:
async with client:
    user = await client.get_me()

print(user.first_name)

Filippo


In [7]:
async with client:
    #channel = await client.get_entity('JakiDale')
    channel = await client.get_entity(1163643200)

    #await client(JoinChannelRequest(channel.id))

print(channel.stringify())

Channel(
	id=1163643200,
	title='Prodigeek Official',
	photo=ChatPhoto(
		photo_id=6023591964085171127,
		dc_id=4,
		has_video=False,
		stripped_thumb=b'\x01\x08\x08\xd5\xfd\xe7\x9aq\xf7}\xe8\xa2\x8a\x05c'
	),
	date=datetime.datetime(2025, 3, 14, 9, 5, tzinfo=datetime.timezone.utc),
	creator=False,
	left=False,
	broadcast=False,
	verified=False,
	megagroup=True,
	restricted=False,
	signatures=False,
	min=False,
	scam=False,
	has_link=True,
	has_geo=False,
	slowmode_enabled=False,
	call_active=False,
	call_not_empty=False,
	fake=False,
	gigagroup=False,
	noforwards=False,
	join_to_send=True,
	join_request=False,
	forum=False,
	stories_hidden=False,
	stories_hidden_min=False,
	stories_unavailable=True,
	signature_profiles=False,
	access_hash=384215503251515819,
	username=None,
	restriction_reason=[
	],
	admin_rights=None,
	banned_rights=None,
	default_banned_rights=ChatBannedRights(
		until_date=datetime.datetime(2038, 1, 19, 3, 14, 7, tzinfo=datetime.timezone.utc),
		view_messages=False

In [8]:
async with client:
    dialogs = await client.get_dialogs()

    for dialog in dialogs:
        if 'prodigeek' in dialog.title.lower():
            dialog_id = dialog.id


print(f"Conversazione con Prodigeek trovata con id {dialog_id}")



Conversazione con Prodigeek trovata con id -1001163643200


In [9]:
# Leggiamo 3000 messaggi

group_messages = dict()
async with client:
    messages = client.iter_messages(entity=channel,limit=3000)
    async for message in messages:
        group_messages[message.id] = message





In [10]:
message_sample = group_messages[list(group_messages.keys())[2]]

print(len(group_messages))
print(message_sample.stringify())

3000
Message(
	id=8286776,
	peer_id=PeerChannel(
		channel_id=1163643200
	),
	date=datetime.datetime(2025, 3, 14, 16, 58, 5, tzinfo=datetime.timezone.utc),
	message='Si regolano in automatico?',
	out=False,
	mentioned=False,
	media_unread=False,
	silent=False,
	post=False,
	from_scheduled=False,
	legacy=False,
	edit_hide=False,
	pinned=False,
	noforwards=False,
	invert_media=False,
	offline=False,
	video_processing_pending=False,
	from_id=PeerUser(
		user_id=1187317641
	),
	from_boosts_applied=None,
	saved_peer_id=None,
	fwd_from=None,
	via_bot_id=None,
	via_business_bot_id=None,
	reply_to=None,
	media=None,
	reply_markup=None,
	entities=[
	],
	views=None,
	forwards=None,
	replies=MessageReplies(
		replies=0,
		replies_pts=9472281,
		comments=False,
		recent_repliers=[
		],
		channel_id=None,
		max_id=None,
		read_max_id=None
	),
	edit_date=None,
	post_author=None,
	grouped_id=None,
	reactions=None,
	restriction_reason=[
	],
	ttl_period=None,
	quick_reply_shortcut_id=None,
	effect=None

In [11]:
# Find all unique users who sent those messages

active_users = set()

for message_id, message in group_messages.items():
    user = message.from_id
    if user is not None and 'user_id' in user.__dict__:
        active_users.add(user.user_id)

print(len(active_users))

310


In [12]:
# Find non-reply messages

non_reply_messages = set()

for message_id, message in group_messages.items():
    if message.reply_to is None:
        non_reply_messages.add(message_id)

print(len(non_reply_messages))

995


In [13]:
# Build a list of (Reply-Message, Source-Message)

replies_list = list()

for message_id, message in group_messages.items():
    if message_id in non_reply_messages:
        continue

    assert message.reply_to is not None

    replied_to_message_id = message.reply_to.reply_to_msg_id

    replies_list.append((message_id, replied_to_message_id))

In [14]:
len(replies_list)

2005

In [61]:
# Build the Network from the Edge List

def author_id_by_message_id(message_id):
    if message_id not in group_messages:
        return None
    message = group_messages[message_id]
    user = message.from_id
    if user is not None and 'user_id' in user.__dict__:
        return user.user_id
    return None

G = nx.DiGraph()
for b, a in replies_list:
    A, B = author_id_by_message_id(a), author_id_by_message_id(b)
    if A and B:
        G.add_edge(B, A)

G.order(), G.size()

(249, 904)

In [62]:
nx.write_gexf(
    G=G,
    path='../../data/prodigeek_telegram.gexf'
)


In [63]:
# Add attributes on edges

for b, a in replies_list:
    A, B = author_id_by_message_id(a), author_id_by_message_id(b)
    if A and B:
        msgA, msgB = group_messages[a], group_messages[b]
        if 'messages' not in G[B][A]:
            G[B][A]['messages'] = list()

        G[B][A]['messages'].append(msgB.message)


In [115]:
# Add attributes on nodes

from telethon.tl.functions.channels import GetParticipantsRequest
from telethon.tl.types import ChannelParticipantsSearch

async with client:
    participants = await client(GetParticipantsRequest(
        -1001163643200, ChannelParticipantsSearch(''), 0, 50,
        hash=0
    ))

for node in G.nodes:
    G.nodes[node]['role'] = ''

for p in participants.participants:
    if p.user_id in G.nodes:
        G.nodes[p.user_id]['role'] = p.rank or ''

In [119]:
# Join list attributes

for edge in G.edges:
    G.edges[edge]['messages'] = ''.join(G.edges[edge]['messages'])

In [126]:
# Add username as attribute to nodes

async def get_username_by_id(user_id):
    async with client:
        user = await client.get_entity(user_id)

    if user.username is not None:
        return user.username
    return user.first_name

for node in G.nodes:
    G.nodes[node]['name'] = await get_username_by_id(node)


In [127]:
nx.write_gexf(
    G=G,
    path='../../data/telegram/prodigeek_telegram.gexf'
)
